<a href="https://colab.research.google.com/github/m-zayed5722/Miscellaneous-Projects/blob/main/DocSage_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install pypdf sentence-transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.3 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()  # upload a .pdf
pdf_path = next(iter(uploaded.keys()))
pdf_path


StopIteration: 

In [3]:
from pypdf import PdfReader

def extract_pdf_text(path: str) -> list[dict]:
    reader = PdfReader(path)
    pages = []
    for i, page in enumerate(reader.pages):
        text = page.extract_text() or ""
        text = " ".join(text.split())  # clean whitespace
        pages.append({"page": i + 1, "text": text})
    return pages

pages = extract_pdf_text(pdf_path)
len(pages), pages[0]["page"], pages[0]["text"][:300]


NameError: name 'pdf_path' is not defined

In [4]:
def chunk_text(pages, chunk_size=900, overlap=150):
    chunks = []
    for p in pages:
        text = p["text"]
        if not text.strip():
            continue
        start = 0
        while start < len(text):
            end = min(start + chunk_size, len(text))
            chunk = text[start:end]
            chunks.append({
                "page": p["page"],
                "chunk_text": chunk,
            })
            start += (chunk_size - overlap)
    return chunks

chunks = chunk_text(pages, chunk_size=900, overlap=150)
len(chunks), chunks[0]["page"], chunks[0]["chunk_text"][:200]


NameError: name 'pages' is not defined

In [5]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

texts = [c["chunk_text"] for c in chunks]
emb = embed_model.encode(texts, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)

dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine similarity since normalized
index.add(emb)

print("Chunks:", len(chunks), "Embedding dim:", dim)

Exception ignored in: <function _xla_gc_callback at 0x7959437ef240>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7959437ef240>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
def retrieve(query: str, k=5):
    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, ids = index.search(q_emb, k)
    results = []
    for score, idx in zip(scores[0], ids[0]):
        if idx == -1:
            continue
        results.append({
            "score": float(score),
            "page": chunks[idx]["page"],
            "text": chunks[idx]["chunk_text"]
        })
    return results

hits = retrieve("What is this document about?", k=5)
[(h["score"], h["page"]) for h in hits]


In [ ]:
import re

def split_sentences(t: str):
    # simple sentence splitter
    s = re.split(r'(?<=[.!?])\s+', t)
    return [x.strip() for x in s if x.strip()]

def answer_grounded(query: str, k=5, max_sentences=6):
    retrieved = retrieve(query, k=k)
    # collect candidate sentences with their page
    candidates = []
    for r in retrieved:
        for sent in split_sentences(r["text"]):
            # score sentence by embedding similarity to query (cheap but effective)
            candidates.append((r["page"], sent))

    # embed sentences in batches
    sent_texts = [s for _, s in candidates]
    if not sent_texts:
        return "I couldn't find relevant text in the PDF.", []

    q_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    s_emb = embed_model.encode(sent_texts, convert_to_numpy=True, normalize_embeddings=True)
    sims = (s_emb @ q_emb.T).reshape(-1)

    top_idx = np.argsort(-sims)[:max_sentences]
    picked = [(candidates[i][0], candidates[i][1], float(sims[i])) for i in top_idx]

    # build answer + citations
    answer_lines = []
    cites = []
    for page, sent, sim in picked:
        answer_lines.append(f"- {sent} (p.{page})")
        cites.append(page)

    # de-duplicate cite pages
    cites = sorted(set(cites))
    final = "Here’s what I found in the document:\n" + "\n".join(answer_lines)
    final += "\n\nCitations: " + ", ".join([f"p.{p}" for p in cites])
    return final, retrieved

ans, retrieved = answer_grounded("Summarize the main points.", k=6, max_sentences=7)
print(ans)


In [ ]:
USE_LLM = False  # flip True if you add an LLM call

def build_context(retrieved, max_chars=4500):
    # Keep context short
    blocks = []
    total = 0
    for r in retrieved:
        block = f"[p.{r['page']}] {r['text']}\n"
        if total + len(block) > max_chars:
            break
        blocks.append(block)
        total += len(block)
    return "\n".join(blocks)

def llm_answer(query: str, retrieved):
    # placeholder — insert your provider call here
    raise NotImplementedError("Add your LLM call here (OpenAI/local/etc.)")

def rag_chat(query: str, k=5):
    retrieved = retrieve(query, k=k)
    context = build_context(retrieved)

    if not retrieved:
        return "I couldn't retrieve relevant passages from the PDF."

    if USE_LLM:
        return llm_answer(query, retrieved)

    # fallback grounded answer
    ans, _ = answer_grounded(query, k=k, max_sentences=7)
    return ans

print(rag_chat("What are the key definitions in this document?", k=6))
